## Load expanded triples from DeepSeek, make it unique triples

In [25]:
from datasets import Dataset

dataset = Dataset.load_from_disk("/projects/JHA/shared/triples/deepseek_brings_about_top20")
print(dataset)
print(dataset[0])

Dataset({
    features: ['input_ids', 'attention_mask', 'positions', 'matched_head_token_ids', 'matched_heads', 'head_token_ids', 'head', 'predictions', 'tails'],
    num_rows: 259860
})
{'input_ids': [[2, 13, 1930, 2706, 1920, 3700, 26, 6478, 8277, 12, 2292, 9418, 16, 6442, 2292, 6934, 1007, 13, 18, 23, 17, 50, 43, 3505, 5339, 2161, 2193, 43, 25, 17, 7464, 4795, 17, 30210, 4778, 2899, 8761, 24578, 3436, 3798, 4682, 1988, 5098, 1956, 1920, 10956, 3957, 1927, 1920, 3214, 2161, 30, 2535, 4280, 4695, 1930, 7122, 16, 6396, 1930, 26670, 16, 5952, 17, 21862, 2859, 1930, 19765, 16, 2659, 1930, 3455, 4990, 16, 3771, 17, 2454, 4695, 16, 1930, 8578, 4695, 37, 3276, 39, 18, 4852, 16, 1920, 24578, 1985, 11214, 1942, 7857, 4695, 7447, 1927, 22255, 31, 2406, 16, 24578, 1985, 3549, 1966, 2436, 3957, 16, 47, 18, 49, 18, 16, 4097, 8979, 16, 1958, 1920, 4032, 2859, 1930, 4767, 15992, 2444, 1930, 2084, 1920, 3]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [31]:
num = 0
a = dataset['tails']
for b in a:
    num += len(b)
print(num)

1633523


In [11]:
dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ['head', 'tails']])
print(dataset)
print(dataset[50])

Dataset({
    features: ['head', 'tails'],
    num_rows: 259860
})
{'head': 'diabetes', 'tails': ['anemia disease', 'benign neoplasm', 'obstructive disease', 'rupture disease', 'congenital disease', 'arthrop disease']}


In [19]:
head_to_tails = {}
for entry in dataset:
    head = entry["head"]
    tails = [tail for tail in entry["tails"] if tail.strip()] 
    
    if head not in head_to_tails:
        head_to_tails[head] = set()  # Use a set to ensure uniqueness

    head_to_tails[head].update(tails)

# Convert to Hugging Face Dataset format
unique_heads_dataset = Dataset.from_dict({
    "head": list(head_to_tails.keys()),
    "tails": [list(tails) for tails in head_to_tails.values()]
})

In [23]:
unique_heads_dataset.save_to_disk("/projects/JHA/shared/triples/deepseek_brings_about_top20_unique")

Saving the dataset (0/1 shards):   0%|          | 0/293 [00:00<?, ? examples/s]

In [32]:
num = 0
for entry in unique_heads_dataset:
    num += len(entry['tails'])
print(num)
print(len(unique_heads_dataset))

148216
293


In [35]:
print(len(unique_heads_dataset[0]['tails']))

16951


## build context for each query